# Libraries

In [367]:
import pandas as pd
import numpy as np

from ethanol_brazil_plants.utils import utils

from datetime import date
from xlrd.xldate import xldate_as_datetime

pd.options.display.max_columns = 100

# Functions

In [368]:
def normalize_renovabio_database(
    raw_df: pd.DataFrame,
    object_cols: list[str],
    int_cols: list[str],
    float_cols: list[str],
    date_cols: list[str],
) -> pd.DataFrame:
    """
    Normalize renovabio database columns
    """

    normalized_df = utils.normalize_header(raw_df)
    normalized_df = normalized_df.fillna(method='ffill')

    normalized_df = utils.normalize_object_columns(normalized_df, object_cols)
    normalized_df = normalized_df[normalized_df['CNPJ'] != '']

    normalized_df = utils.normalize_int_columns(normalized_df, int_cols)
    normalized_df = utils.normalize_float_columns(normalized_df, float_cols)

    normalized_df = utils.normalize_date_columns(normalized_df, date_cols)

    return normalized_df

In [369]:
def generate_razao_cidade_uf(normalized_df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate RAZAO_SOCIAL, CIDADE and UF columns.
    """

    normalized_df['RAZAO_SOCIAL'] = normalized_df['RAZAO_SOCIAL-CIDADE-UF'].str.split('-').str[0]

    normalized_df[['CIDADE', 'UF', 'UF_2']] = (
        normalized_df['RAZAO_SOCIAL-CIDADE-UF']
        .str.split('-')
        .str[-2:]
        .str.join(',')
        .str.split(',', expand=True)
    )

    normalized_df['UF'] = np.where(
        normalized_df['UF'].isnull(),
        normalized_df['UF_2'],
        normalized_df['UF']
    )

    normalized_df['CIDADE'] = np.where(
        normalized_df['RAZAO_SOCIAL'] !=
        normalized_df['CIDADE'],
        normalized_df['CIDADE'],
        None
    )

    normalized_df = normalized_df.drop(['RAZAO_SOCIAL-CIDADE-UF', 'UF_2'], axis=1)
    return normalized_df

# Data

In [370]:
raw_rbp_validos = catalog.load("raw_renovabio_plants_validos")

[08/07/23 10:49:22] INFO     Loading data from 'raw_renovabio_plants_validos' (ExcelDataSet)... ]8;id=922356;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=329913;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

# Normalize

In [371]:
# Normalize
object_cols = [
    'RAZAO_SOCIAL-CIDADE-UF',
    'BIOCOMBUSTIVEL',
    'ROTA',
    'FIRMA_INSPETORA',
    'ENDERECO_EMISSOR_PRIMARIO',
    'CNPJ',
    'PROCESSO_DE_CERTIFICACAO',
]

int_cols = [
    'CNPJ',
    'PROCESSO_DE_CERTIFICACAO']
    
float_cols = [
    'NOTA_DE_EFICIENCIA_ENERGETICO-AMBIENTAL_GCO2EQ/MJ',
    'VOLUME_ELEGIVEL_%',
    'FATOR_PARA_EMISSAO_DE_CBIO_TCO2EQ/L',
]

date_cols = ['DATA_DE_APROVACAO_PELA_ANP', 'VALIDADE']

normalized_rbp_validos = normalize_renovabio_database(
    raw_rbp_validos, object_cols,
    int_cols, float_cols, date_cols,
)

# RAZAO_SOCIAL, CIDADE and UF

In [372]:
normalized_rbp_validos = generate_razao_cidade_uf(normalized_rbp_validos)

# Rota

In [373]:
ds_rota = {
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO CANA-DE-ACUCAR':
    'ETANOL 1G CANA',
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO CANA-DE-ACUCAR E MILHO EM USINA INTEGRADA':
    'ETANOL 1G CANA E MILHO',
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO MILHO':
    'ETANOL 1G MILHO',
    'ETANOL COMBUSTIVEL DE PRIMEIRA E SEGUNDA GERACAO PRODUZIDO EM USINA INTEGRADA':
    'ETANOL 1G E 2G',
}

cd_rota = {
    'BIODIESEL': 1,
    'BIOMETANO': 2,
    'ETANOL 1G E 2G': 3,
    'ETANOL 1G CANA': 4,
    'ETANOL 1G CANA E MILHO': 5,
    'ETANOL 1G MILHO': 6,
}

normalized_rbp_validos['DS_ROTA'] = normalized_rbp_validos['ROTA'].replace(ds_rota)
normalized_rbp_validos['CD_ROTA'] = normalized_rbp_validos['DS_ROTA'].replace(cd_rota)

# Location

## ZIP Code

In [374]:
normalized_rbp_validos['CEP'] = (
    normalized_rbp_validos['ENDERECO_EMISSOR_PRIMARIO']
    .str.replace('.','')
    .str.replace(r'-(\s+)', '-', regex=True)
    .str.findall(r'\d{5}-\d{3}|\d{5}-\d{2}')
    .str[0]
    .str.replace('-','')
    .str.pad(8, side='right', fillchar='0')
)

# City and State

In [375]:
normalized_rbp_validos['DS_END'] = (
    normalized_rbp_validos['ENDERECO_EMISSOR_PRIMARIO']
    .str.replace(r'S/N[\w\W]| NO ', '', regex= True)
    .str.replace(r'CEP[\W]', '', regex=True)
    .str.replace('.', '')
    .str.replace(', ,', ',')
    .str.replace(r'\s+', ' ', regex= True)
)

In [376]:
normalized_rbp_validos[['re_CIDADE', 're_UF']] = (
    normalized_rbp_validos['DS_END']
    .str.extract(r'([A-Z\s]+)(?: - |/| / )([A-Z]{2})\b')
)

In [377]:
for col in ['CIDADE', 'UF']:
    normalized_rbp_validos[col] = (
        normalized_rbp_validos[col]
        .fillna(normalized_rbp_validos[f're_{col}'])
    )
    normalized_rbp_validos = normalized_rbp_validos.drop(f're_{col}', axis=1)

# Address

In [378]:
(
    normalized_rbp_validos['DS_END']
).unique()


array(['FAZENDA VISTA ALEGRE PINHAL, ITAPETININGA - SP, 18209-600',
       'FAZENDA SANTA CLARA - 45807-000 - SANTA CRUZ CABRALIA/BA',
       'FAZENDA SAO MATHEUS, ZONA RURAL, 57265-000, TEOTONIO VILELA, AL',
       'FAZENDA SAO JOSE, 57250-000 CAMPO ALEGRE/AL',
       'ROD HOMERO SEVERO LINS, KM 535 SP284 MAIS 6KM ACESSO PART 19500-000 BAIRRO LARANJA DOCE, MARTINOPOLIS - SP',
       'RODOVIA PE 41, KM 10,7, AREA RURAL 53659-899 - IGARASSU/PE NETO/BA',
       'BR 285, KM 179, UNIDADE 2 MUITOS CAPOES/RS 95230-000',
       'ENGENHO CONCEICAO VELA - ZONA RURAL, IPOJUCA - PE 55590-000 NETO/BA',
       'R ODILON NEGRAO, 585 - JARDIM MARIA M CASTRO, ITAPOLIS - SP, 14900-000',
       'FAZENDA MASSAYO, - ZONA RURAL - JUAZEIRO - BA - BRASIL',
       'VILA COLONIA PINDORAMA, 57230-000 - CORURIPE/AL',
       'RODOVIA ANHANGUERA, KM 209 - CAIXA POSTAL: 273, BAIRRO PIRES 13630-970 PIRASSUNUNGA/SP',
       'ZONA RURAL 59170-000',
       'ROD PROFESSOR RICARDO DA COSTA PINTO KM 22, AGUA BRANCA/ ZONA

In [384]:
# Pattern 1 - Highway or roadway
normalized_rbp_validos[['p1_DS_END', 'p1_NO_END']] = (
    normalized_rbp_validos['DS_END']
    .str.extract(r'(^RO[\w\s\d]+|^EST[\w\s\d]+|RODOVIA[\w\s\d]+)[\w\W\s]+(KM [\d]+)')
)

# Pattern 2 - Farm
normalized_rbp_validos[['p2_DS_END']] = (
    normalized_rbp_validos['DS_END']
    .str.extract(r'(^FAZENDA[\w\s]+)')
)

normalized_rbp_validos[['p2_NO_END']] = 'SN'

# Pattern 3 - Street
normalized_rbp_validos[['p3_DS_END', 'p3_NO_END']] = (
    normalized_rbp_validos['DS_END']
    .str.extract(r'(^R[\s][\w\s]+|^RUA[\w\s]+)(?:, |[\W]|,)([\d]+)')
)

# Addresses
for col in ['DS_END', 'NO_END']:
    normalized_rbp_validos[col] = (
        normalized_rbp_validos[f'p1_{col}']
        .fillna(normalized_rbp_validos[f'p2_{col}'])
        .fillna(normalized_rbp_validos[f'p3_{col}'])
    )
    normalized_rbp_validos = normalized_rbp_validos.drop(
        [f'p1_{col}', f'p2_{col}', f'p3_{col}'], axis=1)

In [386]:
normalized_rbp_validos[normalized_rbp_validos['DS_END'].isna()]['ENDERECO_EMISSOR_PRIMARIO'].unique()


array(['ROD HOMERO SEVERO LINS, KM 535 SP284 S/N MAIS 6KM ACESSO PART CEP 19.500-000 BAIRRO LARANJA DOCE, MARTINOPOLIS - SP',
       'RODOVIA PE 41, KM 10,7, S/N- AREA RURAL. CEP: 53.659-899 - IGARASSU/PE NETO/BA',
       'BR 285, KM 179, UNIDADE 2 MUITOS CAPOES/RS CEP: 95230-000',
       'ENGENHO CONCEICAO VELA S/N - ZONA RURAL, IPOJUCA - PE CEP: 55590-000 NETO/BA',
       'R. ODILON NEGRAO, 585 - JARDIM MARIA M CASTRO, ITAPOLIS - SP, 14900-000',
       'VILA COLONIA PINDORAMA, S/NO. CEP: 57.230-000 - CORURIPE/AL',
       'RODOVIA ANHANGUERA, KM 209 - CAIXA POSTAL: 273, BAIRRO PIRES. CEP: 13.630-970. PIRASSUNUNGA/SP',
       'ZONA RURAL S/NO CEP 59.170-000',
       'ROD PROFESSOR RICARDO DA COSTA PINTO KM 22, S/N AGUA BRANCA/ ZONA RURAL ALDEIAS ALTAS/MA',
       'ESTRADA VICINAL ASTORGA/JAGUAPITA, S/NO, KM 21, BAIRRO SANTA ZELIA, MUNICIPIO DE ASTORGA PR. CEP: 86.730-000',
       'BR-369, JANDAIA DO SUL - PR, 86900-000',
       'RUA N, 1844, LT 80 A 85, QD. IND. 07 - DISTRITO INDUSTRI